# Building a Database for Crime Reports

## Introduction

In this project, we will build a database for storing data related with crimes that occurred in Boston. This dataset is available in the file [`crime.csv`](https://www.kaggle.com/AnalyzeBoston/crimes-in-boston). The first rows are shown here:

![nn](img/preview.png)

The first column represents the identifier of the crime. The second contains a numeric identifier code for the committed crime. The third represents a description of the crime. The next two rows contain the date on which the crime happened and the corresponding day of the week. Finally, the last two columns represent the location of the crime with a latitude and longitude coordinates.

The goal of this guided project is for you to create a database named `crimes_db` with a table – `boston_crimes` – with appropriate datatypes for storing the data from the `crime.csv` file. You will be creating the table inside a schema named `crimes`. You will also create the `readonly` and `readwrite` groups with the appropriate privileges as we have discussed in this course. Finally, you will also need to create one user for each of these groups.

The following diagram illustrates a high level overview of what we want to achieve:

![nn](img/goal.png)

## Creating the Crime Database

We will start by creating a database for storing our crime data as well as a schema for containing the tables.

The following diagram illustrates what the database should look like after this screen. We should have a new database named crime_db and a schema in it named crimes.

![nn](img/create_db.png)

In [1]:
import psycopg2
conn = psycopg2.connect(dbname='postgres', user='postgres', password='abc123')
cur = conn.cursor()
conn.autocommit = True
cur.execute('CREATE DATABASE crime_db;')
conn.close()

conn = psycopg2.connect(dbname='crime_db', user='postgres', password='abc123')
cur = conn.cursor()
conn.autocommit = True
cur.execute('CREATE SCHEMA crimes;')

## Obtaining the Column Names and Sample

We now have a database and a schema — we are ready to start creating tables. Before we do that, let's gather some data about our crime dataset so that we can more easily select the right datatypes to use in our table.

Let's start by reading the column names from the `crime.csv` file as well as the first row. In this way we will have them at hand throughout this project so that we can easily take a look at them at any moment.

In [2]:
import csv
with open('dataset/crime.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

## Creating an Auxiliary Function

We now have access to the column headers and the first row of data. This will help us when we create the table as we will not have to remember the names of the columns and using the first row of data we can easily recall what kind of data is stored in each column.

Before we create a table for storing the crime data, we need to identify the proper datatypes for the columns. To help us with that, let's create a function — `get_col_value_set()` — that given the name of a CSV file and a column index (starting a 0) that computes a Python set with all distinct values contained in that column.

This function will be useful for two reasons:
1. Checking whether an enumerated datatype might be a good choice for representing a column.
2. Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns.

In [3]:
def get_col_value_set(csv_filename, col_index):
    import csv
    col_value_set = set()
    with open(csv_filename) as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            col_value_set.add(row[col_index])
    return col_value_set

for col_index in range(len(col_headers)):
    print(col_headers[col_index], len(get_col_value_set('dataset/crime.csv', col_index)))

incident_number 319073
offense_code 222
description 244
date 1177
day_of_the_week 7
lat 18179
long 18179


## Finding the Maximum Length

With the function `get_col_value_set()` we can compute the number of distinct values for each column. Columns with a low number of distinct values tend to be good candidates for enumerated datatypes. 

Another important aspect is to know the longest word in any column containing textual data. We can actually use the previous function for computing this:
1. Use the `get_col_value_set()` function to get the distinct values of that column into a variable named col_values
2. Iterate over the elements of col_values and find the one which is the longest

There are two textual column in the data set, namely, the `description` and `day_of_the_week` columns. However the day of the week contains only 7 values, one for each day. We can tell that the longest of them is `Wednesday` without needing any computation.

Let's compute the maximum length of each value in the `description` column.

In [4]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [5]:
max_len_description = 0
for description in get_col_value_set('dataset/crime.csv', 2):
    max_len_description = max(max_len_description, len(description))
        
print(max_len_description)

58


## Creating the Table

Let's create a table for storing the Boston crime data.

In [6]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


For the `incident_number`, we can use the type `INTEGER` and set it as the primary key. The same datatype can also be used to represent the `offense_code`. 

Since the `description` has at most 58 character we decided to use the datatype `VARCHAR2(100)` for representing it. This leaves some margin while not being so big that we can waste a lot of memory. 

The `date` can be represented as the `DATE` datatype. 

We will create an enumerated datatype named `weekday` for the `day_of_the_week` since there are only 7 possible values.

Finally, for the `lat` and `long` we can use `DECIMAL` datatypes.

In [7]:
cur.execute('''
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday')
''')
cur.execute('''
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code    INTEGER,
        description     VARCHAR(100),
        date            DATE,
        day_of_the_week WEEKDAY,
        lat             DECIMAL,
        long            DECIMAL
    );
''')

## Loading the Data

Now that we have created the table, we can load the data into it. 

After this step, the database should look like this:

![nn](img/table_created.png)

In [8]:
with open('dataset/crime.csv') as file:
    cur.copy_expert('COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;', file)

## Revoking Public Privileges

Now it is time to handle users. Our goal is to create the two user groups that we have learned about: `readonly` and `readwrite`. By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the `public` group and on the `public` schema.

In [9]:
cur.execute('''
    REVOKE ALL ON SCHEMA public FROM public;
''')
cur.execute('''
    REVOKE ALL ON DATABASE crime_db FROM public;
''')

## Creating User Groups

The next step is to create our two user groups. The `readonly` group is supposed to only have privileges to perform `SELECT` queries. In contrast, we want the `readwrite` group to be able to perform `SELECT`, `INSERT`, `DELETE` and `UPDATE` queries.

In [10]:
# Create two groups named readonly and readwrite with the no NOLOGIN option.
cur.execute('''
    CREATE GROUP readonly NOLOGIN;
''')
cur.execute('''
    CREATE GROUP readwrite NOLOGIN;
''')

# Grant CONNECT to the crime_db to both groups.
cur.execute('''
    GRANT CONNECT ON DATABASE crime_db TO readonly;
''')
cur.execute('''
    GRANT CONNECT ON DATABASE crime_db TO readwrite;
''')

# Grant USAGE to the crimes schema to both groups.
cur.execute('''
    GRANT USAGE ON SCHEMA crimes TO readonly;
''')
cur.execute('''
    GRANT USAGE ON SCHEMA crimes TO readwrite;
''')

# Grant group specific privileges to each group on all tables in the crimes schema.
cur.execute('''
    GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;
''')
cur.execute('''
    GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;
''')

## Creating Users

The only thing that we need to do is create users. Right now, the high-level picture of our database looks like this:

![nn](img/user_groups.png)

Let's create one user in each group.

In [11]:
cur.execute('''
    CREATE USER data_analyst WITH PASSWORD 'secret1';
''')
cur.execute('''
    GRANT readonly TO data_analyst;
''')
cur.execute('''
    CREATE USER data_scientist WITH PASSWORD 'secret2';
''')
cur.execute('''
    GRANT readwrite TO data_scientist;
''')

## Testing

Our database should now look like this:

![nn](img/goal.png)

We can use SQL queries to check whether the objects have been created and that users and groups have the right privileges. This requires you to know the Postgres internal tables. We can query the `pg_roles` table to inspect privileges related to the database and the `information_schema.table_privileges` table to inspect table privileges.

In [12]:
# close the old connection to test with a brand new connection
conn.close()

conn = psycopg2.connect(dbname='crime_db', user='postgres', password='abc123')
cur = conn.cursor()
# check users and groups
cur.execute('''
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
''')
for user in cur:
    print(user)
print()
# check privileges
cur.execute('''
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
''')
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
